In [1]:
import tensorflow as tf
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import ndimage
from skimage.transform import resize

2025-07-15 15:57:11.433964: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-07-15 15:57:11.468606: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-07-15 15:57:11.468627: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-07-15 15:57:11.469446: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-07-15 15:57:11.476037: I tensorflow/core/platform/cpu_feature_guar

In [2]:
# import the trained model

model = tf.keras.models.load_model("../../Spring_2024/Bayes_for_comps/TS_bayes_implementation_for_TN/models/trained_gmp_model_dense_32_new.h5")

2025-07-15 15:57:13.487141: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1929] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 31134 MB memory:  -> device: 0, name: Tesla V100S-PCIE-32GB, pci bus id: 0000:06:00.0, compute capability: 7.0


In [3]:
model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, None, None, 3)]   0         
                                                                 
 conv2d (Conv2D)             (None, None, None, 32)    896       
                                                                 
 activation (Activation)     (None, None, None, 32)    0         
                                                                 
 conv2d_1 (Conv2D)           (None, None, None, 32)    9248      
                                                                 
 activation_1 (Activation)   (None, None, None, 32)    0         
                                                                 
 max_pooling2d (MaxPooling2  (None, None, None, 32)    0         
 D)                                                              
                                                           

In [4]:
# Define the feature extractor model

# feature extractor input
feat_ext_input = model.input

# feature extractor output - do this at the ReLu activation layer - as this will give the same features as the dropout layer (It does not matter if it is the dropout or the activation layer, the extracted features will be the same)
feat_ext_output = model.layers[-4].output

feature_extractor_model = tf.keras.models.Model(inputs = feat_ext_input, outputs = feat_ext_output)

In [5]:
feature_extractor_model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, None, None, 3)]   0         
                                                                 
 conv2d (Conv2D)             (None, None, None, 32)    896       
                                                                 
 activation (Activation)     (None, None, None, 32)    0         
                                                                 
 conv2d_1 (Conv2D)           (None, None, None, 32)    9248      
                                                                 
 activation_1 (Activation)   (None, None, None, 32)    0         
                                                                 
 max_pooling2d (MaxPooling2  (None, None, None, 32)    0         
 D)                                                              
                                                             

In [6]:
# The following functions will help create all the required matrices

In [7]:
def chose_xml_and_jpeg(file_location):
    # list all files in location
    list_of_all_files = os.listdir(file_location)
    # sort files
    list_of_all_files.sort()
    # separate xml and jpeg files
    all_xml_files = [file for file in list_of_all_files if file.split('.')[-1] == 'xml']
    all_xml_files.sort()
    all_jpeg_files = [file for file in list_of_all_files if file not in all_xml_files]
    all_jpeg_files.sort()
    # get the final 7 image files
    chosen_jpeg_files = all_jpeg_files[-7:]
    
    return chosen_jpeg_files

In [8]:
def create_sub_windows(folder_path, file, stride = 8, kernel_size = 32):
    # joined image path
    joined_im_path = os.path.join(folder_path, file)
    # read the image
    loaded_im_file = plt.imread(joined_im_path)
    # create subwindows and get prediction
    img_height = loaded_im_file.shape[0]
    img_width = loaded_im_file.shape[1]

    # catch all subwindows here
    all_subwindows = []
    # you can also keep track the subwindows here if required - but let's not worry about that for now
    for i in  range(0, img_height, stride):
        for j in range(0, img_width, stride):
            sub_window = loaded_im_file[i: i + kernel_size, j : j + kernel_size,:]
            # resize the subwindow - for 300*300
            sub_window = resize(sub_window, (kernel_size, kernel_size,3))
            # append these to the list
            all_subwindows.append(sub_window)
            
    return all_subwindows  

In [9]:
def extract_features(file_name, file_path):
    # get the subwindows
    subwindows = create_sub_windows(file_path, file_name, 8, 32)
    # stack the subwindows
    stacked_subwindows = np.stack(subwindows, axis = 0)
    # print the shape of this
    print(stacked_subwindows.shape)
    # extract features
    extracted_featrues = feature_extractor_model.predict(stacked_subwindows)
    
    return extracted_featrues

In [10]:
# Let's first take care of the train and validation data

In [11]:
# block 0101

In [12]:
block_0101 = "../../Spring_2024/S_lab_TasselNet/Block_1_TN/Block_1_images_and_xml"

In [13]:
images_block_0101 = chose_xml_and_jpeg(block_0101)

In [14]:
images_block_0101

['Block0101_2020_08_26.jpeg',
 'Block0101_2020_08_27.jpeg',
 'Block0101_2020_08_28.jpeg',
 'Block0101_2020_08_31.jpeg',
 'Block0101_2020_09_02.jpeg',
 'Block0101_2020_09_07.jpeg',
 'Block0101_2020_09_16.jpeg']

In [15]:
%%time
# get the extracted input features

blk_0101_all_features = []
for file in images_block_0101:
    extracted_features = extract_features(file, block_0101)
    blk_0101_all_features.append(extracted_features)

(12288, 32, 32, 3)


2025-07-15 15:57:17.576110: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8907


384/384 [==============================] - 1s 2ms/step
(12288, 32, 32, 3)
384/384 [==============================] - 1s 2ms/step
(12288, 32, 32, 3)
384/384 [==============================] - 1s 2ms/step
(12288, 32, 32, 3)
384/384 [==============================] - 1s 2ms/step
(12288, 32, 32, 3)
384/384 [==============================] - 1s 2ms/step
(12288, 32, 32, 3)
384/384 [==============================] - 1s 2ms/step
(12288, 32, 32, 3)
384/384 [==============================] - 1s 2ms/step
CPU times: user 28.7 s, sys: 1.72 s, total: 30.4 s
Wall time: 29.3 s


In [16]:
# stack the extracted features
blk_0101_stacked_extracted_features = np.stack(blk_0101_all_features, axis = 1)

In [17]:
blk_0101_stacked_extracted_features.shape

(12288, 7, 32)

In [18]:
# see if the mean of the extracted features match to the ones earlier
np.mean(blk_0101_stacked_extracted_features)

0.37572336

In [19]:
# Let's not save this for now, let's save the final stacks of train and validation data

In [20]:
# block 0102

In [21]:
block_0102 = "../../Spring_2024/S_lab_TasselNet/Block_2_TN/Block_2_images_and_xml"

In [22]:
images_block_0102 = chose_xml_and_jpeg(block_0102)

In [23]:
images_block_0102

['Block0102_2020_08_26.jpeg',
 'Block0102_2020_08_27.jpeg',
 'Block0102_2020_08_28.jpeg',
 'Block0102_2020_08_31.jpeg',
 'Block0102_2020_09_02.jpeg',
 'Block0102_2020_09_07.jpeg',
 'Block0102_2020_09_16.jpeg']

In [24]:
%%time
# get the extracted input features

blk_0102_all_features = []
for file in images_block_0102:
    extracted_features = extract_features(file, block_0102)
    blk_0102_all_features.append(extracted_features)

(12288, 32, 32, 3)
384/384 [==============================] - 1s 2ms/step
(12288, 32, 32, 3)
384/384 [==============================] - 1s 2ms/step
(12288, 32, 32, 3)
384/384 [==============================] - 1s 2ms/step
(12288, 32, 32, 3)
384/384 [==============================] - 1s 2ms/step
(12288, 32, 32, 3)
384/384 [==============================] - 1s 2ms/step
(12288, 32, 32, 3)
384/384 [==============================] - 1s 2ms/step
(12288, 32, 32, 3)
384/384 [==============================] - 1s 2ms/step
CPU times: user 28.4 s, sys: 1.35 s, total: 29.8 s
Wall time: 28.5 s


In [25]:
# stack the extracted features
blk_0102_stacked_extracted_features = np.stack(blk_0102_all_features, axis = 1)

In [26]:
blk_0102_stacked_extracted_features.shape

(12288, 7, 32)

In [27]:
# see if the mean of the extracted features match to the ones earlier
np.mean(blk_0102_stacked_extracted_features)

0.37976986

In [28]:
# block 0203

In [29]:
block_0203 = "../../Spring_2024/S_lab_TasselNet/Block_9_TN/Block_9_images_and_xml"

In [30]:
images_block_0203 = chose_xml_and_jpeg(block_0203)

In [31]:
images_block_0203

['Block0203_2020_08_26.jpeg',
 'Block0203_2020_08_27.jpeg',
 'Block0203_2020_08_28.jpeg',
 'Block0203_2020_08_31.jpeg',
 'Block0203_2020_09_02.jpeg',
 'Block0203_2020_09_07.jpeg',
 'Block0203_2020_09_16.jpeg']

In [32]:
%%time
# get the extracted input features

blk_0203_all_features = []
for file in images_block_0203:
    extracted_features = extract_features(file, block_0203)
    blk_0203_all_features.append(extracted_features)

(12288, 32, 32, 3)
384/384 [==============================] - 1s 2ms/step
(12288, 32, 32, 3)
384/384 [==============================] - 1s 2ms/step
(12288, 32, 32, 3)
384/384 [==============================] - 1s 2ms/step
(12288, 32, 32, 3)
384/384 [==============================] - 1s 2ms/step
(12288, 32, 32, 3)
384/384 [==============================] - 1s 2ms/step
(12288, 32, 32, 3)
384/384 [==============================] - 1s 2ms/step
(12288, 32, 32, 3)
384/384 [==============================] - 1s 2ms/step
CPU times: user 28.3 s, sys: 1.34 s, total: 29.7 s
Wall time: 28.4 s


In [33]:
# stack the extracted features
blk_0203_stacked_extracted_features = np.stack(blk_0203_all_features, axis = 1)

In [34]:
blk_0203_stacked_extracted_features.shape

(12288, 7, 32)

In [35]:
# see if the mean of the extracted features match to the ones earlier
np.mean(blk_0203_stacked_extracted_features)

0.3369076

In [36]:
# block 0301

In [37]:
block_0301 = "../../Spring_2024/S_lab_TasselNet/Block_13_TN/Block_13_images_and_xml"

In [38]:
images_block_0301 = chose_xml_and_jpeg(block_0301)

In [39]:
images_block_0301

['Block0301_2020_08_26.jpeg',
 'Block0301_2020_08_27.jpeg',
 'Block0301_2020_08_28.jpeg',
 'Block0301_2020_08_31.jpeg',
 'Block0301_2020_09_02.jpeg',
 'Block0301_2020_09_07.jpeg',
 'Block0301_2020_09_16.jpeg']

In [40]:
%%time
# get the extracted input features

blk_0301_all_features = []
for file in images_block_0301:
    extracted_features = extract_features(file, block_0301)
    blk_0301_all_features.append(extracted_features)

(12288, 32, 32, 3)
384/384 [==============================] - 1s 2ms/step
(12288, 32, 32, 3)
384/384 [==============================] - 1s 2ms/step
(12288, 32, 32, 3)
384/384 [==============================] - 1s 2ms/step
(12288, 32, 32, 3)
384/384 [==============================] - 1s 2ms/step
(12288, 32, 32, 3)
384/384 [==============================] - 1s 2ms/step
(12288, 32, 32, 3)
384/384 [==============================] - 1s 2ms/step
(12288, 32, 32, 3)
384/384 [==============================] - 1s 2ms/step
CPU times: user 28.2 s, sys: 1.4 s, total: 29.6 s
Wall time: 28.5 s


In [41]:
# stack the extracted features
blk_0301_stacked_extracted_features = np.stack(blk_0301_all_features, axis = 1)

In [42]:
blk_0301_stacked_extracted_features.shape

(12288, 7, 32)

In [43]:
# see if the mean of the extracted features match to the ones earlier
np.mean(blk_0301_stacked_extracted_features)

0.3708836

In [44]:
# We can stack all these now and save?

all_train_targets_list = [blk_0101_stacked_extracted_features, blk_0102_stacked_extracted_features, blk_0203_stacked_extracted_features, blk_0301_stacked_extracted_features]

In [45]:
# stack all these together?
all_train_targets = np.vstack(all_train_targets_list)

In [46]:
all_train_targets.shape

(49152, 7, 32)

In [47]:
# sanity check
print(np.mean(all_train_targets[12288*0:12288*1, :, :] == blk_0101_stacked_extracted_features))
print(np.mean(all_train_targets[12288*1:12288*2, :, :] == blk_0102_stacked_extracted_features))
print(np.mean(all_train_targets[12288*2:12288*3, :, :] == blk_0203_stacked_extracted_features))
print(np.mean(all_train_targets[12288*3:12288*4, :, :] == blk_0301_stacked_extracted_features))

1.0
1.0
1.0
1.0


In [48]:
# save the stack
np.save("data/Overlapping_data/Train_features_and_targets/all_train_targets.npy", all_train_targets)

In [49]:
# sanity check
np.mean(all_train_targets == np.load("data/Overlapping_data/Train_features_and_targets/all_train_targets.npy"))

1.0

In [50]:
# Do this also for the validation data, and then stop for the day?

In [51]:
# block 0204

In [52]:
block_0204 = "../../Spring_2024/S_lab_TasselNet/Block_10_TN/Block_10_images_and_xml"

In [53]:
images_block_0204 = chose_xml_and_jpeg(block_0204)

In [54]:
images_block_0204

['Block0204_2020_08_26.jpeg',
 'Block0204_2020_08_27.jpeg',
 'Block0204_2020_08_28.jpeg',
 'Block0204_2020_08_31.jpeg',
 'Block0204_2020_09_02.jpeg',
 'Block0204_2020_09_07.jpeg',
 'Block0204_2020_09_16.jpeg']

In [55]:
%%time
# get the extracted input features

blk_0204_all_features = []
for file in images_block_0204:
    extracted_features = extract_features(file, block_0204)
    blk_0204_all_features.append(extracted_features)

(12288, 32, 32, 3)
384/384 [==============================] - 1s 2ms/step
(12288, 32, 32, 3)
384/384 [==============================] - 1s 2ms/step
(12288, 32, 32, 3)
384/384 [==============================] - 1s 2ms/step
(12288, 32, 32, 3)
384/384 [==============================] - 1s 2ms/step
(12288, 32, 32, 3)
384/384 [==============================] - 1s 2ms/step
(12288, 32, 32, 3)
384/384 [==============================] - 1s 2ms/step
(12288, 32, 32, 3)
384/384 [==============================] - 1s 2ms/step
CPU times: user 28.3 s, sys: 1.47 s, total: 29.7 s
Wall time: 28.6 s


In [56]:
# stack the extracted features
blk_0204_stacked_extracted_features = np.stack(blk_0204_all_features, axis = 1)

In [57]:
blk_0204_stacked_extracted_features.shape

(12288, 7, 32)

In [58]:
# see if the mean of the extracted features match to the ones earlier
np.mean(blk_0204_stacked_extracted_features)

0.37136865

In [59]:
# save the data
np.save("data/Overlapping_data/Valid_features_and_targets/all_valid_targets.npy", blk_0204_stacked_extracted_features)

In [60]:
# sanity check
np.mean(blk_0204_stacked_extracted_features == np.load("data/Overlapping_data/Valid_features_and_targets/all_valid_targets.npy"))

1.0

Preprocessing for the test data

In [61]:
# Block 0103

In [62]:
block_0103 = "../../Spring_2024/S_lab_TasselNet/Block_3_TN/Block_3_images_and_xml"

In [63]:
images_block_0103 = chose_xml_and_jpeg(block_0103)

In [64]:
images_block_0103

['Block0103_2020_08_26.jpeg',
 'Block0103_2020_08_27.jpeg',
 'Block0103_2020_08_28.jpeg',
 'Block0103_2020_08_31.jpeg',
 'Block0103_2020_09_02.jpeg',
 'Block0103_2020_09_07.jpeg',
 'Block0103_2020_09_16.jpeg']

In [65]:
%%time
# get the extracted input features

blk_0103_all_features = []
for file in images_block_0103:
    extracted_features = extract_features(file, block_0103)
    blk_0103_all_features.append(extracted_features)

(12288, 32, 32, 3)
384/384 [==============================] - 1s 2ms/step
(12288, 32, 32, 3)
384/384 [==============================] - 1s 2ms/step
(12288, 32, 32, 3)
384/384 [==============================] - 1s 2ms/step
(12288, 32, 32, 3)
384/384 [==============================] - 1s 2ms/step
(12288, 32, 32, 3)
384/384 [==============================] - 1s 2ms/step
(12288, 32, 32, 3)
384/384 [==============================] - 1s 2ms/step
(12288, 32, 32, 3)
384/384 [==============================] - 1s 2ms/step
CPU times: user 28.2 s, sys: 1.44 s, total: 29.6 s
Wall time: 28.5 s


In [66]:
# stack the extracted features
blk_0103_stacked_extracted_features = np.stack(blk_0103_all_features, axis = 1)

In [67]:
blk_0103_stacked_extracted_features.shape

(12288, 7, 32)

In [68]:
# see if the mean of the extracted features match to the ones earlier
np.mean(blk_0103_stacked_extracted_features)

0.3638021

In [69]:
# save the data
np.save("data/Overlapping_data/Test_out_targets/test_targets_blk_0103.npy", blk_0103_stacked_extracted_features)

In [70]:
# sanity check
np.mean(blk_0103_stacked_extracted_features == np.load("data/Overlapping_data/Test_out_targets/test_targets_blk_0103.npy"))

1.0

In [71]:
# Block 0104

In [72]:
block_0104 = "../../Spring_2024/S_lab_TasselNet/Block_4_TN/Block_4_images_and_xml"

In [73]:
images_block_0104 = chose_xml_and_jpeg(block_0104)

In [74]:
images_block_0104

['Block0104_2020_08_26.jpeg',
 'Block0104_2020_08_27.jpeg',
 'Block0104_2020_08_28.jpeg',
 'Block0104_2020_08_31.jpeg',
 'Block0104_2020_09_02.jpeg',
 'Block0104_2020_09_07.jpeg',
 'Block0104_2020_09_16.jpeg']

In [75]:
%%time
# get the extracted input features

blk_0104_all_features = []
for file in images_block_0104:
    extracted_features = extract_features(file, block_0104)
    blk_0104_all_features.append(extracted_features)

(12288, 32, 32, 3)
384/384 [==============================] - 1s 2ms/step
(12288, 32, 32, 3)
384/384 [==============================] - 1s 2ms/step
(12288, 32, 32, 3)
384/384 [==============================] - 1s 2ms/step
(12288, 32, 32, 3)
384/384 [==============================] - 1s 2ms/step
(12288, 32, 32, 3)
384/384 [==============================] - 1s 2ms/step
(12288, 32, 32, 3)
384/384 [==============================] - 1s 2ms/step
(12288, 32, 32, 3)
384/384 [==============================] - 1s 2ms/step
CPU times: user 28.5 s, sys: 1.57 s, total: 30.1 s
Wall time: 28.9 s


In [76]:
# stack the extracted features
blk_0104_stacked_extracted_features = np.stack(blk_0104_all_features, axis = 1)

In [77]:
blk_0104_stacked_extracted_features.shape

(12288, 7, 32)

In [78]:
# see if the mean of the extracted features match to the ones earlier
np.mean(blk_0104_stacked_extracted_features)

0.41262174

In [79]:
# save the data
np.save("data/Overlapping_data/Test_out_targets/test_targets_blk_0104.npy", blk_0104_stacked_extracted_features)

In [80]:
# sanity check
np.mean(blk_0104_stacked_extracted_features == np.load("data/Overlapping_data/Test_out_targets/test_targets_blk_0104.npy"))

1.0

In [81]:
# Block 0105

In [82]:
block_0105 = "../../Spring_2024/S_lab_TasselNet/Block_5_TN/Block_5_images_and_xml"

In [83]:
images_block_0105 = chose_xml_and_jpeg(block_0105)

In [84]:
images_block_0105

['Block0105_2020_08_26.jpeg',
 'Block0105_2020_08_27.jpeg',
 'Block0105_2020_08_28.jpeg',
 'Block0105_2020_08_31.jpeg',
 'Block0105_2020_09_02.jpeg',
 'Block0105_2020_09_07.jpeg',
 'Block0105_2020_09_16.jpeg']

In [85]:
%%time
# get the extracted input features

blk_0105_all_features = []
for file in images_block_0105:
    extracted_features = extract_features(file, block_0105)
    blk_0105_all_features.append(extracted_features)

(12288, 32, 32, 3)
384/384 [==============================] - 1s 2ms/step
(12288, 32, 32, 3)
384/384 [==============================] - 1s 2ms/step
(12288, 32, 32, 3)
384/384 [==============================] - 1s 2ms/step
(12288, 32, 32, 3)
384/384 [==============================] - 1s 2ms/step
(12288, 32, 32, 3)
384/384 [==============================] - 1s 2ms/step
(12288, 32, 32, 3)
384/384 [==============================] - 1s 2ms/step
(12288, 32, 32, 3)
384/384 [==============================] - 1s 2ms/step
CPU times: user 28.4 s, sys: 1.44 s, total: 29.9 s
Wall time: 28.6 s


In [86]:
# stack the extracted features
blk_0105_stacked_extracted_features = np.stack(blk_0105_all_features, axis = 1)

In [87]:
blk_0105_stacked_extracted_features.shape

(12288, 7, 32)

In [88]:
# see if the mean of the extracted features match to the ones earlier
np.mean(blk_0105_stacked_extracted_features)

0.3939796

In [89]:
# save the data
np.save("data/Overlapping_data/Test_out_targets/test_targets_blk_0105.npy", blk_0105_stacked_extracted_features)

In [90]:
# sanity check
np.mean(blk_0105_stacked_extracted_features == np.load("data/Overlapping_data/Test_out_targets/test_targets_blk_0105.npy"))

1.0

In [91]:
# Block 0106

In [92]:
block_0106 = "../../Spring_2024/S_lab_TasselNet/Block_6_TN/Block_6_images_and_xml"

In [93]:
images_block_0106 = chose_xml_and_jpeg(block_0106)

In [94]:
images_block_0106

['Block0106_2020_08_26.jpeg',
 'Block0106_2020_08_27.jpeg',
 'Block0106_2020_08_28.jpeg',
 'Block0106_2020_08_31.jpeg',
 'Block0106_2020_09_02.jpeg',
 'Block0106_2020_09_07.jpeg',
 'Block0106_2020_09_16.jpeg']

In [95]:
%%time
# get the extracted input features

blk_0106_all_features = []
for file in images_block_0106:
    extracted_features = extract_features(file, block_0106)
    blk_0106_all_features.append(extracted_features)

(12288, 32, 32, 3)
384/384 [==============================] - 1s 2ms/step
(12288, 32, 32, 3)
384/384 [==============================] - 1s 2ms/step
(12288, 32, 32, 3)
384/384 [==============================] - 1s 2ms/step
(12288, 32, 32, 3)
384/384 [==============================] - 1s 2ms/step
(12288, 32, 32, 3)
384/384 [==============================] - 1s 2ms/step
(12288, 32, 32, 3)
384/384 [==============================] - 1s 2ms/step
(12288, 32, 32, 3)
384/384 [==============================] - 1s 2ms/step
CPU times: user 28.4 s, sys: 1.5 s, total: 29.9 s
Wall time: 28.7 s


In [96]:
# stack the extracted features
blk_0106_stacked_extracted_features = np.stack(blk_0106_all_features, axis = 1)

In [97]:
blk_0106_stacked_extracted_features.shape

(12288, 7, 32)

In [98]:
# see if the mean of the extracted features match to the ones earlier
np.mean(blk_0106_stacked_extracted_features)

0.40003976

In [99]:
# save the data
np.save("data/Overlapping_data/Test_out_targets/test_targets_blk_0106.npy", blk_0106_stacked_extracted_features)

In [100]:
# sanity check
np.mean(blk_0106_stacked_extracted_features == np.load("data/Overlapping_data/Test_out_targets/test_targets_blk_0106.npy"))

1.0

In [101]:
# Block 0201

In [102]:
block_0201 = "../../Spring_2024/S_lab_TasselNet/Block_7_TN/Block_7_images_and_xml"

In [103]:
images_block_0201 = chose_xml_and_jpeg(block_0201)

In [104]:
images_block_0201

['Block0201_2020_08_26.jpeg',
 'Block0201_2020_08_27.jpeg',
 'Block0201_2020_08_28.jpeg',
 'Block0201_2020_08_31.jpeg',
 'Block0201_2020_09_02.jpeg',
 'Block0201_2020_09_07.jpeg',
 'Block0201_2020_09_16.jpeg']

In [105]:
%%time
# get the extracted input features

blk_0201_all_features = []
for file in images_block_0201:
    extracted_features = extract_features(file, block_0201)
    blk_0201_all_features.append(extracted_features)

(12288, 32, 32, 3)
384/384 [==============================] - 1s 2ms/step
(12288, 32, 32, 3)
384/384 [==============================] - 1s 2ms/step
(12288, 32, 32, 3)
384/384 [==============================] - 1s 2ms/step
(12288, 32, 32, 3)
384/384 [==============================] - 1s 2ms/step
(12288, 32, 32, 3)
384/384 [==============================] - 1s 2ms/step
(12288, 32, 32, 3)
384/384 [==============================] - 1s 2ms/step
(12288, 32, 32, 3)
384/384 [==============================] - 1s 2ms/step
CPU times: user 28.4 s, sys: 1.47 s, total: 29.9 s
Wall time: 28.7 s


In [106]:
# stack the extracted features
blk_0201_stacked_extracted_features = np.stack(blk_0201_all_features, axis = 1)

In [107]:
blk_0201_stacked_extracted_features.shape

(12288, 7, 32)

In [108]:
# see if the mean of the extracted features match to the ones earlier
np.mean(blk_0201_stacked_extracted_features)

0.36519957

In [109]:
# save the data
np.save("data/Overlapping_data/Test_out_targets/test_targets_blk_0201.npy", blk_0201_stacked_extracted_features)

In [110]:
# sanity check
np.mean(blk_0201_stacked_extracted_features == np.load("data/Overlapping_data/Test_out_targets/test_targets_blk_0201.npy"))

1.0

In [111]:
# Block 0202

In [112]:
block_0202 = "../../Spring_2024/S_lab_TasselNet/Block_8_TN/Block_8_images_and_xml"

In [113]:
images_block_0202 = chose_xml_and_jpeg(block_0202)

In [114]:
images_block_0202

['Block0202_2020_08_26.jpeg',
 'Block0202_2020_08_27.jpeg',
 'Block0202_2020_08_28.jpeg',
 'Block0202_2020_08_31.jpeg',
 'Block0202_2020_09_02.jpeg',
 'Block0202_2020_09_07.jpeg',
 'Block0202_2020_09_16.jpeg']

In [115]:
%%time
# get the extracted input features

blk_0202_all_features = []
for file in images_block_0202:
    extracted_features = extract_features(file, block_0202)
    blk_0202_all_features.append(extracted_features)

(12288, 32, 32, 3)
384/384 [==============================] - 1s 2ms/step
(12288, 32, 32, 3)
384/384 [==============================] - 1s 2ms/step
(12288, 32, 32, 3)
384/384 [==============================] - 1s 2ms/step
(12288, 32, 32, 3)
384/384 [==============================] - 1s 2ms/step
(12288, 32, 32, 3)
384/384 [==============================] - 1s 2ms/step
(12288, 32, 32, 3)
384/384 [==============================] - 1s 2ms/step
(12288, 32, 32, 3)
384/384 [==============================] - 1s 2ms/step
CPU times: user 28.3 s, sys: 1.5 s, total: 29.8 s
Wall time: 28.6 s


In [116]:
# stack the extracted features
blk_0202_stacked_extracted_features = np.stack(blk_0202_all_features, axis = 1)

In [117]:
blk_0202_stacked_extracted_features.shape

(12288, 7, 32)

In [118]:
# see if the mean of the extracted features match to the ones earlier
np.mean(blk_0202_stacked_extracted_features)

0.33197743

In [119]:
# save the data
np.save("data/Overlapping_data/Test_out_targets/test_targets_blk_0202.npy", blk_0202_stacked_extracted_features)

In [120]:
# sanity check
np.mean(blk_0202_stacked_extracted_features == np.load("data/Overlapping_data/Test_out_targets/test_targets_blk_0202.npy"))

1.0

In [121]:
# Block 0205

In [122]:
block_0205 = "../../Spring_2024/S_lab_TasselNet/Block_11_TN/Block_11_images_and_xml"

In [123]:
images_block_0205 = chose_xml_and_jpeg(block_0205)

In [124]:
images_block_0205

['Block0205_2020_08_26.jpeg',
 'Block0205_2020_08_27.jpeg',
 'Block0205_2020_08_28.jpeg',
 'Block0205_2020_08_31.jpeg',
 'Block0205_2020_09_02.jpeg',
 'Block0205_2020_09_07.jpeg',
 'Block0205_2020_09_16.jpeg']

In [125]:
%%time
# get the extracted input features

blk_0205_all_features = []
for file in images_block_0205:
    extracted_features = extract_features(file, block_0205)
    blk_0205_all_features.append(extracted_features)

(12288, 32, 32, 3)
384/384 [==============================] - 1s 2ms/step
(12288, 32, 32, 3)
384/384 [==============================] - 1s 2ms/step
(12288, 32, 32, 3)
384/384 [==============================] - 1s 2ms/step
(12288, 32, 32, 3)
384/384 [==============================] - 1s 2ms/step
(12288, 32, 32, 3)
384/384 [==============================] - 1s 2ms/step
(12288, 32, 32, 3)
384/384 [==============================] - 1s 2ms/step
(12288, 32, 32, 3)
384/384 [==============================] - 1s 2ms/step
CPU times: user 28.6 s, sys: 1.45 s, total: 30 s
Wall time: 28.8 s


In [126]:
# stack the extracted features
blk_0205_stacked_extracted_features = np.stack(blk_0205_all_features, axis = 1)

In [127]:
blk_0205_stacked_extracted_features.shape

(12288, 7, 32)

In [128]:
# see if the mean of the extracted features match to the ones earlier
np.mean(blk_0205_stacked_extracted_features)

0.40675408

In [129]:
# save the data
np.save("data/Overlapping_data/Test_out_targets/test_targets_blk_0205.npy", blk_0205_stacked_extracted_features)

In [130]:
# sanity check
np.mean(blk_0205_stacked_extracted_features == np.load("data/Overlapping_data/Test_out_targets/test_targets_blk_0205.npy"))

1.0

In [131]:
# Block 0206

In [132]:
block_0206 = "../../Spring_2024/S_lab_TasselNet/Block_12_TN/Block_12_images_and_xml"

In [133]:
images_block_0206 = chose_xml_and_jpeg(block_0206)

In [134]:
images_block_0206

['Block0206_2020_08_26.jpeg',
 'Block0206_2020_08_27.jpeg',
 'Block0206_2020_08_28.jpeg',
 'Block0206_2020_08_31.jpeg',
 'Block0206_2020_09_02.jpeg',
 'Block0206_2020_09_07.jpeg',
 'Block0206_2020_09_16.jpeg']

In [135]:
%%time
# get the extracted input features

blk_0206_all_features = []
for file in images_block_0206:
    extracted_features = extract_features(file, block_0206)
    blk_0206_all_features.append(extracted_features)

(12288, 32, 32, 3)
384/384 [==============================] - 1s 2ms/step
(12288, 32, 32, 3)
384/384 [==============================] - 1s 2ms/step
(12288, 32, 32, 3)
384/384 [==============================] - 1s 2ms/step
(12288, 32, 32, 3)
384/384 [==============================] - 1s 2ms/step
(12288, 32, 32, 3)
384/384 [==============================] - 1s 2ms/step
(12288, 32, 32, 3)
384/384 [==============================] - 1s 2ms/step
(12288, 32, 32, 3)
384/384 [==============================] - 1s 2ms/step
CPU times: user 28.3 s, sys: 1.51 s, total: 29.8 s
Wall time: 28.6 s


In [136]:
# stack the extracted features
blk_0206_stacked_extracted_features = np.stack(blk_0206_all_features, axis = 1)

In [137]:
blk_0206_stacked_extracted_features.shape

(12288, 7, 32)

In [138]:
# see if the mean of the extracted features match to the ones earlier
np.mean(blk_0206_stacked_extracted_features)

0.3615067

In [139]:
# save the data
np.save("data/Overlapping_data/Test_out_targets/test_targets_blk_0206.npy", blk_0206_stacked_extracted_features)

In [140]:
# sanity check
np.mean(blk_0206_stacked_extracted_features == np.load("data/Overlapping_data/Test_out_targets/test_targets_blk_0206.npy"))

1.0

In [141]:
# Block 0302

In [142]:
block_0302 = "../../Spring_2024/S_lab_TasselNet/Block_14_TN/Block_14_images_and_xml"

In [143]:
images_block_0302 = chose_xml_and_jpeg(block_0302)

In [144]:
images_block_0302

['Block0302_2020_08_26.jpeg',
 'Block0302_2020_08_27.jpeg',
 'Block0302_2020_08_28.jpeg',
 'Block0302_2020_08_31.jpeg',
 'Block0302_2020_09_02.jpeg',
 'Block0302_2020_09_07.jpeg',
 'Block0302_2020_09_16.jpeg']

In [145]:
%%time
# get the extracted input features

blk_0302_all_features = []
for file in images_block_0302:
    extracted_features = extract_features(file, block_0302)
    blk_0302_all_features.append(extracted_features)

(12288, 32, 32, 3)
384/384 [==============================] - 1s 2ms/step
(12288, 32, 32, 3)
384/384 [==============================] - 1s 2ms/step
(12288, 32, 32, 3)
384/384 [==============================] - 1s 2ms/step
(12288, 32, 32, 3)
384/384 [==============================] - 1s 2ms/step
(12288, 32, 32, 3)
384/384 [==============================] - 1s 2ms/step
(12288, 32, 32, 3)
384/384 [==============================] - 1s 2ms/step
(12288, 32, 32, 3)
384/384 [==============================] - 1s 2ms/step
CPU times: user 28.3 s, sys: 1.46 s, total: 29.8 s
Wall time: 28.6 s


In [146]:
# stack the extracted features
blk_0302_stacked_extracted_features = np.stack(blk_0302_all_features, axis = 1)

In [147]:
blk_0302_stacked_extracted_features.shape

(12288, 7, 32)

In [148]:
# see if the mean of the extracted features match to the ones earlier
np.mean(blk_0302_stacked_extracted_features)

0.3624974

In [149]:
# save the data
np.save("data/Overlapping_data/Test_out_targets/test_targets_blk_0302.npy", blk_0302_stacked_extracted_features)

In [150]:
# sanity check
np.mean(blk_0302_stacked_extracted_features == np.load("data/Overlapping_data/Test_out_targets/test_targets_blk_0302.npy"))

1.0

In [151]:
# Block 0303

In [152]:
block_0303 = "../../Spring_2024/S_lab_TasselNet/Block_15_TN/Block_15_images_and_xml"

In [153]:
images_block_0303 = chose_xml_and_jpeg(block_0303)

In [154]:
images_block_0303

['Block0303_2020_08_26.jpeg',
 'Block0303_2020_08_27.jpeg',
 'Block0303_2020_08_28.jpeg',
 'Block0303_2020_08_31.jpeg',
 'Block0303_2020_09_02.jpeg',
 'Block0303_2020_09_07.jpeg',
 'Block0303_2020_09_16.jpeg']

In [155]:
%%time
# get the extracted input features

blk_0303_all_features = []
for file in images_block_0303:
    extracted_features = extract_features(file, block_0303)
    blk_0303_all_features.append(extracted_features)

(12288, 32, 32, 3)
384/384 [==============================] - 1s 2ms/step
(12288, 32, 32, 3)
384/384 [==============================] - 1s 2ms/step
(12288, 32, 32, 3)
384/384 [==============================] - 1s 2ms/step
(12288, 32, 32, 3)
384/384 [==============================] - 1s 2ms/step
(12288, 32, 32, 3)
384/384 [==============================] - 1s 2ms/step
(12288, 32, 32, 3)
384/384 [==============================] - 1s 2ms/step
(12288, 32, 32, 3)
384/384 [==============================] - 1s 2ms/step
CPU times: user 28.2 s, sys: 1.28 s, total: 29.5 s
Wall time: 28.3 s


In [156]:
# stack the extracted features
blk_0303_stacked_extracted_features = np.stack(blk_0303_all_features, axis = 1)

In [157]:
blk_0303_stacked_extracted_features.shape

(12288, 7, 32)

In [158]:
# see if the mean of the extracted features match to the ones earlier
np.mean(blk_0303_stacked_extracted_features)

0.35323527

In [159]:
# save the data
np.save("data/Overlapping_data/Test_out_targets/test_targets_blk_0303.npy", blk_0303_stacked_extracted_features)

In [160]:
# sanity check
np.mean(blk_0303_stacked_extracted_features == np.load("data/Overlapping_data/Test_out_targets/test_targets_blk_0303.npy"))

1.0

In [161]:
# Block 0304

In [162]:
block_0304 = "../../Spring_2024/S_lab_TasselNet/Block_16_TN/Block_16_images_and_xml"

In [163]:
images_block_0304 = chose_xml_and_jpeg(block_0304)

In [164]:
images_block_0304

['Block0304_2020_08_26.jpeg',
 'Block0304_2020_08_27.jpeg',
 'Block0304_2020_08_28.jpeg',
 'Block0304_2020_08_31.jpeg',
 'Block0304_2020_09_02.jpeg',
 'Block0304_2020_09_07.jpeg',
 'Block0304_2020_09_16.jpeg']

In [165]:
%%time
# get the extracted input features

blk_0304_all_features = []
for file in images_block_0304:
    extracted_features = extract_features(file, block_0304)
    blk_0304_all_features.append(extracted_features)

(12288, 32, 32, 3)
384/384 [==============================] - 1s 2ms/step
(12288, 32, 32, 3)
384/384 [==============================] - 1s 2ms/step
(12288, 32, 32, 3)
384/384 [==============================] - 1s 2ms/step
(12288, 32, 32, 3)
384/384 [==============================] - 1s 2ms/step
(12288, 32, 32, 3)
384/384 [==============================] - 1s 2ms/step
(12288, 32, 32, 3)
384/384 [==============================] - 1s 2ms/step
(12288, 32, 32, 3)
384/384 [==============================] - 1s 2ms/step
CPU times: user 28.3 s, sys: 1.34 s, total: 29.6 s
Wall time: 28.4 s


In [166]:
# stack the extracted features
blk_0304_stacked_extracted_features = np.stack(blk_0304_all_features, axis = 1)

In [167]:
blk_0304_stacked_extracted_features.shape

(12288, 7, 32)

In [168]:
# see if the mean of the extracted features match to the ones earlier
np.mean(blk_0304_stacked_extracted_features)

0.38126844

In [169]:
# save the data
np.save("data/Overlapping_data/Test_out_targets/test_targets_blk_0304.npy", blk_0304_stacked_extracted_features)

In [170]:
# sanity check
np.mean(blk_0304_stacked_extracted_features == np.load("data/Overlapping_data/Test_out_targets/test_targets_blk_0304.npy"))

1.0

In [171]:
# Block 0305

In [172]:
block_0305 = "../../Spring_2024/S_lab_TasselNet/Block_17_TN/Block_17_images_and_xml"

In [173]:
images_block_0305 = chose_xml_and_jpeg(block_0305)

In [174]:
images_block_0305

['Block0305_2020_08_26.jpeg',
 'Block0305_2020_08_27.jpeg',
 'Block0305_2020_08_28.jpeg',
 'Block0305_2020_08_31.jpeg',
 'Block0305_2020_09_02.jpeg',
 'Block0305_2020_09_07.jpeg',
 'Block0305_2020_09_16.jpeg']

In [175]:
%%time
# get the extracted input features

blk_0305_all_features = []
for file in images_block_0305:
    extracted_features = extract_features(file, block_0305)
    blk_0305_all_features.append(extracted_features)

(12288, 32, 32, 3)
384/384 [==============================] - 1s 2ms/step
(12288, 32, 32, 3)
384/384 [==============================] - 1s 2ms/step
(12288, 32, 32, 3)
384/384 [==============================] - 1s 2ms/step
(12288, 32, 32, 3)
384/384 [==============================] - 1s 2ms/step
(12288, 32, 32, 3)
384/384 [==============================] - 1s 2ms/step
(12288, 32, 32, 3)
384/384 [==============================] - 1s 2ms/step
(12288, 32, 32, 3)
384/384 [==============================] - 1s 2ms/step
CPU times: user 28.4 s, sys: 1.39 s, total: 29.8 s
Wall time: 28.5 s


In [176]:
# stack the extracted features
blk_0305_stacked_extracted_features = np.stack(blk_0305_all_features, axis = 1)

In [177]:
blk_0305_stacked_extracted_features.shape

(12288, 7, 32)

In [178]:
# see if the mean of the extracted features match to the ones earlier
np.mean(blk_0305_stacked_extracted_features)

0.36241043

In [179]:
# save the data
np.save("data/Overlapping_data/Test_out_targets/test_targets_blk_0305.npy", blk_0305_stacked_extracted_features)

In [180]:
# sanity check
np.mean(blk_0305_stacked_extracted_features == np.load("data/Overlapping_data/Test_out_targets/test_targets_blk_0305.npy"))

1.0

In [181]:
# Block 0306

In [182]:
block_0306 = "../../Spring_2024/S_lab_TasselNet/Block_18_TN/Block_18_images_and_xml"

In [183]:
images_block_0306 = chose_xml_and_jpeg(block_0306)

In [184]:
images_block_0306

['Block0306_2020_08_26.jpeg',
 'Block0306_2020_08_27.jpeg',
 'Block0306_2020_08_28.jpeg',
 'Block0306_2020_08_31.jpeg',
 'Block0306_2020_09_02.jpeg',
 'Block0306_2020_09_07.jpeg',
 'Block0306_2020_09_16.jpeg']

In [185]:
%%time
# get the extracted input features

blk_0306_all_features = []
for file in images_block_0306:
    extracted_features = extract_features(file, block_0306)
    blk_0306_all_features.append(extracted_features)

(12288, 32, 32, 3)
384/384 [==============================] - 1s 2ms/step
(12288, 32, 32, 3)
384/384 [==============================] - 1s 2ms/step
(12288, 32, 32, 3)
384/384 [==============================] - 1s 2ms/step
(12288, 32, 32, 3)
384/384 [==============================] - 1s 2ms/step
(12288, 32, 32, 3)
384/384 [==============================] - 1s 2ms/step
(12288, 32, 32, 3)
384/384 [==============================] - 1s 2ms/step
(12288, 32, 32, 3)
384/384 [==============================] - 1s 2ms/step
CPU times: user 28.4 s, sys: 1.32 s, total: 29.7 s
Wall time: 28.5 s


In [186]:
# stack the extracted features
blk_0306_stacked_extracted_features = np.stack(blk_0306_all_features, axis = 1)

In [187]:
blk_0306_stacked_extracted_features.shape

(12288, 7, 32)

In [188]:
# see if the mean of the extracted features match to the ones earlier
np.mean(blk_0306_stacked_extracted_features)

0.3712025

In [189]:
# save the data
np.save("data/Overlapping_data/Test_out_targets/test_targets_blk_0306.npy", blk_0306_stacked_extracted_features)

In [190]:
# sanity check
np.mean(blk_0306_stacked_extracted_features == np.load("data/Overlapping_data/Test_out_targets/test_targets_blk_0306.npy"))

1.0